<a href="https://colab.research.google.com/github/Th4End/TP_ANN_Titanic/blob/main/FouilleDonnee/ANN/TP_ANN_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : Classification Binaire avec PyTorch – Titanic

## **Objectif du TP**
Prédire si un passager du Titanic a **survécu ou non** à partir de ses caractéristiques (âge, sexe, classe, etc.) en utilisant un **réseau de neurones fully-connected** avec **PyTorch**.

---

## **Compétences abordées**

| Thème | Détail |
|------|--------|
| **Préparation des données** | Chargement, nettoyage, split train/dev |
| **PyTorch Dataset & DataLoader** | Gestion efficace des batchs |
| **Modèle personnalisé** | `nn.Module`, couches cachées, dropout, ReLU |
| **Boucle d'entraînement** | `optimizer.zero_grad()`, `loss.backward()` |
| **Évaluation** | Accuracy, CrossEntropyLoss |
| **Soumission Kaggle** | Format CSV |
| **(Bonus)** | Grid Search + Validation Croisée |

---

## **Consignes**
- Complétez **chaque cellule de code** en suivant les instructions en **Markdown**.
- Les **imports sont fournis**.
- Testez à chaque étape !
- **Bonus optionnel** en fin de TP.
- Vous pouvez interroger un LLM pour déboguer votre modèle, mais vous devez expliquer les modifications proposées et pourquoi vous les acceptez ou non.
---

**Bon courage !**

## **0. Imports (déjà fournis)**

Exécutez cette cellule **sans modification**.

In [13]:
import numpy as np
import polars as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import csv
import os

# Device (GPU si disponible)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device utilisé : {DEVICE}")

Device utilisé : cpu


## **1. Hyperparamètres**

Définissez les hyperparamètres du modèle. Vous les modifierez plus tard pour améliorer les performances.


In [14]:
# MODIFIEZ CES VALEURS POUR AMÉLIORER LE MODÈLE !
HIDDEN_SIZE = ...      # Ex: 50
NUM_LAYERS = ...       # Ex: 3 (nombre de couches cachées)
DROPOUT_PROB = ...     # Ex: 0.2
LEARNING_RATE = ...    # Ex: 0.01
NUM_EPOCHS = ...       # Ex: 50
BATCH_SIZE = ...       # Ex: 50

## **2. Chargement et préparation des données**

- Chargez `train_clean.csv` et `test_clean.csv`
- Extrayez :
  - `X` : toutes les colonnes **à partir de la 3e** (index 2)
  - `y` : la colonne **Survived** (index 1)
- Convertissez en `np.float32` pour `X` et `np.int64` pour `y`
- Séparez en `train` (90%) et `dev` (10%) avec `stratify=y` et `random_state=42`

**À compléter**

In [15]:
df_californiaHousing = pl.read_csv("/content/sample_data/california_housing_test.csv")
df_californiaHousingtrain = pl.read_csv("/content/sample_data/california_housing_train.csv")
print(df_californiaHousing)
print(df_californiaHousingtrain)

shape: (3_000, 9)
┌───────────┬──────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ longitude ┆ latitude ┆ housing_m ┆ total_roo ┆ … ┆ populatio ┆ household ┆ median_in ┆ median_ho │
│ ---       ┆ ---      ┆ edian_age ┆ ms        ┆   ┆ n         ┆ s         ┆ come      ┆ use_value │
│ f64       ┆ f64      ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│           ┆          ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ -122.05   ┆ 37.37    ┆ 27.0      ┆ 3885.0    ┆ … ┆ 1537.0    ┆ 606.0     ┆ 6.6085    ┆ 344700.0  │
│ -118.3    ┆ 34.26    ┆ 43.0      ┆ 1510.0    ┆ … ┆ 809.0     ┆ 277.0     ┆ 3.599     ┆ 176500.0  │
│ -117.81   ┆ 33.78    ┆ 27.0      ┆ 3589.0    ┆ … ┆ 1484.0    ┆ 495.0     ┆ 5.7934    ┆ 270500.0  │
│ -118.36   ┆ 33.82    ┆ 28.0      ┆ 67.0      ┆ … ┆ 49.0      ┆ 11.0    

## **3. Dataset PyTorch**

Créez une classe `TitanicDataset` qui hérite de `torch.utils.data.Dataset`.

Elle doit :
- Accepter `X` et `y` (optionnel)
- Convertir les données en tenseurs PyTorch
- Implémenter `__len__` et `__getitem__`

**À compléter**

In [18]:
class TitanicDataset(Dataset):
  def __init__(self, X, y=None):
    self.X = torch.tensor(X, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.int64) if y is not None else None
    self.n_samples = self.X.shape[0]

    def __len__(self):
      return self.n_samples

  def __getitem__(self, index):
    if self.y is not None:
      return self.X[index], self.y[index]
    else:
      return self.X[index]

## **4. DataLoaders**

Créez trois `DataLoader` :
- `train_loader` → `shuffle=True`
- `dev_loader` → `shuffle=False`
- `test_loader` → données de test (sans `y`)

**À compléter**

## **5. Modèle : Réseau de neurones**

Créez une classe `TitanicModel` avec :
- `num_layers` couches cachées de taille `hidden_size`
- Activation **ReLU**
- **Dropout** après chaque couche cachée
- Sortie : 2 classes (utilisez `nn.Linear` final)
- Utilisez `nn.Sequential` pour construire le réseau

**À compléter**

## **6. Fonction de coût et optimiseur**

- `criterion` : `CrossEntropyLoss`
- `optimizer` : `Adam` avec le `learning_rate` défini

**À compléter**

## **7. Boucle d'entraînement (1 epoch)**

Écrivez une fonction `train_epoch` qui :
- Met le modèle en mode `.train()`
- Parcourt les batchs
- Calcule la perte, fait `backward()` et `step()`
- Retourne la **perte moyenne**

**À compléter**

## **8. Évaluation**

Écrivez une fonction `evaluate` qui :
- Met le modèle en `.eval()`
- Désactive le gradient (`torch.no_grad()`)
- Calcule l'**accuracy** sur le dataset de validation

**À compléter**

## **9. Entraînement complet**

Boucle sur `NUM_EPOCHS` :
- Appel à `train_epoch`
- Toutes les 10 epochs : afficher l'accuracy sur `dev`

**À compléter**

## **10. Prédiction sur le test & Soumission Kaggle (Optionnel)**

- Passez le modèle en `.eval()`
- Parcourez `test_loader`
- Récupérez les prédictions avec `torch.max(..., 1)`
- Générez `submission.csv` avec `PassengerId` et `Survived`

**À compléter**

## **EXERCICE BONUS (OPTIONNEL)**

### **Grid Search + Validation Croisée**

> **Objectif** : Trouver automatiquement les meilleurs hyperparamètres.

**À faire (seulement si vous avez fini !)**

1. Utilisez `KFold` pour diviser les données
2. Testez plusieurs valeurs de :
   - `learning_rate`
   - `dropout_prob`
   - `hidden_size`
3. Entraînez un modèle pour chaque combinaison
4. Gardez le modèle avec la **meilleure accuracy moyenne**

**Indice** : Utilisez `ParameterGrid` de `sklearn`

---

**Question** : Pourquoi le Grid Search est-il utile ? Donnez un exemple de compromis entre `dropout` élevé et sous-apprentissage.

In [ ]:
# Votre code Grid Search ici (optionnel)
